In [1]:
import importlib
from glob import glob
import numpy as np
import pandas as pd
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/stacked_lstm_statefull/utils.py'>

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Bidirectional
from keras.preprocessing.sequence import pad_sequences
# from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
word2ind, ind2word = utils.load_index_word_map()

In [5]:
vocab_size = len(word2ind)

In [6]:
batch_size = 0

while batch_size < 500:
    train_filename = np.random.choice(glob('data/train_*.csv'), size=1)[0]
    before_scan = pd.read_csv(train_filename)
    batch_size = before_scan.shape[0]
batch_size

In [8]:
def scanning(array, input_length):
    batches = []
    text_length = array.shape[1]
    for i in range(text_length-input_length):
        batches.append(array[:, i:i+input_length+1])
    return np.concatenate(batches)

In [9]:
emb_dim = 128
input_length = 300

In [10]:
dataset = scanning(before_scan.values, input_length)

In [11]:
dataset.shape

(347571, 301)

In [12]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(batch_size, input_length)))
model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Dropout(.2))
model.add(Bidirectional(LSTM(50, activation='softmax', return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Dropout(.2))
model.add(Bidirectional(LSTM(100, activation='softmax', dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Dropout(.2))
model.add(Dense(vocab_size, activation='softmax'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (613, 300, 128)           12800     
_________________________________________________________________
conv1d_1 (Conv1D)            (613, 300, 256)           131328    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (613, 150, 256)           0         
_________________________________________________________________
dropout_1 (Dropout)          (613, 150, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (613, 150, 100)           122800    
_________________________________________________________________
batch_normalization_1 (Batch (613, 150, 100)           400       
_________________________________________________________________
conv1d_2 (Conv1D)            (613, 150, 256)           102656    
__________

In [14]:
model.set_weights(load_model('model_alter.h5').get_weights())

In [15]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
for _ in range(2):
    print(_)
    model.reset_states()
    model_hist = model.fit(dataset[:, :-1], dataset[:, [-1]], epochs=1, batch_size=batch_size)
    if model_hist.history['acc'][0] > .55: break

0
Epoch 1/1
347571/347571 [==============================] - 276s - loss: 1.8291 - acc: 0.4638   
1
Epoch 1/1
347571/347571 [==============================] - 274s - loss: 1.7730 - acc: 0.4753   


In [17]:
# save model
model.save('model_alter.h5')

In [18]:
pred_model = Sequential()
pred_model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(1, input_length)))
pred_model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Dropout(.2))
pred_model.add(Bidirectional(LSTM(50, activation='softmax', return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Dropout(.2))
pred_model.add(Bidirectional(LSTM(100, activation='softmax', dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Dropout(.2))
pred_model.add(Dense(vocab_size, activation='softmax'))

In [19]:
pred_model.set_weights(model.get_weights())

In [20]:
def implement(seed_text, maxlen=100, must_stop=2000, n_likely=5):
    cleaned = list(seed_text)
    padded_input_tokens = utils.tokenise(word2ind, cleaned)
    res_tokens = [token for token in padded_input_tokens]
    pred_model.reset_states()
    while must_stop > 0:
        padded_input_tokens = pad_sequences([padded_input_tokens], maxlen=maxlen)
        probs = pred_model.predict(padded_input_tokens, batch_size=1)[0]
        first_n = np.argsort(probs)[-n_likely:]
        probs = probs[first_n] / np.sum(probs[first_n])
        predicted = np.random.choice(first_n, p=probs)
        padded_input_tokens = padded_input_tokens[0].tolist()
        padded_input_tokens.append(predicted)
        res_tokens.append(predicted)
        must_stop -= 1
    detokenised = utils.detokenise(ind2word, res_tokens)
    return ''.join(detokenised)

In [21]:
res = implement('We are friends, good friends', 300, 600, 2)

In [23]:
print(res)

 lolllllloolrlrIroooolololloolo-oroorrrrrrorolrorrr
o
roorloolrooloollooolollolollolroo-r-o-roo-roooolrlooorooorror OOr OIo
EIIIrIrrIEI
IEEEIEEEEEEIIIrrEEI
EIIIrIIIIIIrEIEIIrIIIEEEIIErEIEIEEIEIIIrIIIEIEEIIIIIEIEIEIIrEEIEIIIrEEEIIEEIIIIEEIIIIIIrIIIIIIIIIIIIIIIIIEEIEIEIIIIIIEIIIEIIIIEIIEIEIIIIIIIIEIIEIIIIIIIIEIIIEIIEEIEEIEIEEIEIEEEUEEENENNEEEENEEEEEENENEEEEPEEEEPEPPEEPE
PPQPEPEEQPPPEPP
QPPPPPPPE
PQPQPQQQPQQPPQQQPPQQQPPQPQPPQQQPQQQQQQPPPQQQPQQQQN           )   )           )  )          )          ))         )        )))       )    )  )      )  )   )  )   )    )        )              
